# 7. Interaction between Excel

xlwings is best demonstrated with an Excel workbook open at the same time

> ***xlwings version:*** *The latest xlwings version at the time of writing this document is 0.18.0.*

## Alternative to VBA

The primary selling point for *xlwings* is to provide and alternative to Excel's built-in programming language VBA. The best thing about this is that is enables the user to take advantage of Python's large ecosystem of both built-in language features and third party packages like *Pandas*, *Numpy*, *Scipy*, *Matplotlib* etc.


## Installation

Install the *xlwings* add-in by

```code
pip install xlwings
```

Or if you use the Anaconda distribution

```code
conda install xlwings
```


## Initial setup

### Create an Excel/Python file pair

This is easiest done by opening the Command Prompt in the directory you want to have the files and running 

```code
xlwings quickstart file_name
```
This should create two files called `file_name.xlsm` and `file_name.py`, which provides an easier setup for the work.


> ***Hint:*** *You can open the Command Promt (cmd) in a specific folder in multiple ways:*
1. *Open cmd and change directory by `cd path_to_directory`. Get the directory path by copying from file explorer.*
2. *Open cmd directly from file explorer by typing `cmd` in the address line.*


### Install the *xlwings* add-in for Excel
    
1. Go to the [xlwings releases on GitHub](https://github.com/xlwings/xlwings/releases) and download the file called `xlwings.xlam` from the release matching your installed version of *xlwings*. You can check your version of *xlwings* by `pip show xlwings`. 
  
  
2. Once downloaded, go to Excel => Developer tab => Excel Add-ins => browse for the `xlwings.xlam` file you just downloaded. Pressing "OK" should load the *xlwings* add-in as a ribbon tab in Excel. 
        
*See [here](https://docs.xlwings.org/en/stable/addin.html#installation) for the installation instructions from the official documentation.*

## The two created files

**TODO**


## Connecting to a workbook from Python

When we work with the *xlwings* API we deal with objects. This is common for many Python packages.

Internally in *xlwings*, a ***workbook*** is implemented as class. Let's try to create an instance of that class to connect to our newly created workbook.

In [1]:
import xlwings as xw

In [2]:
# Connect to workbook
wb = xw.Book('Session 7 - Interaction with Excel.xlsm')

# Print the class instance
print(wb)

# And the type of the instance
print(type(wb))

<Book [Session 7 - Interaction with Excel.xlsm]>
<class 'xlwings.main.Book'>


## Referring to a sheet

In [35]:

# Extract the sheets from the workbook
shts = wb.sheets

# Let's inspect what `shts` actually is
print(shts)
print('')
print(type(shts))

Sheets([<Sheet [Session 7 - Interaction with Excel.xlsm]Sheet1>, <Sheet [Session 7 - Interaction with Excel.xlsm]Sheet2>, <Sheet [Session 7 - Interaction with Excel.xlsm]Sheet3>])

<class 'xlwings.main.Sheets'>


Note that the above refers to all the sheets in the workbook. It is seen that our workbook has three sheets, i.e. `Sheet1`, `Sheet2` and `Sheet3`.

If we want a single sheet, say `Sheet1`, we can extract it directly 

In [36]:
# Extract only specific sheet
sht = shts['Sheet1']

# Print to inspect
print(type(sht))
print(sht)

<class 'xlwings.main.Sheet'>
<Sheet [Session 7 - Interaction with Excel.xlsm]Sheet1>


## Extract values from Excel to Python

Extracting values from Excel to Python can be done by

**TODO**

## Inserting values from Python to Excel

Values can be inserted into a sheet from Python by

In [37]:
sht['A1'].value = 1

This might seem quite insignificant at a glance, but is this concept can be used to bring everything that Python offers into Excel.

A parameter computed with Python can be inserted into the Excel sheet in the same manner

In [38]:
sht['A2'].value = list(range(10))

Notice that the list is inserted horizontally by default. To insert is vertically instead, do this:

In [39]:
sht['A3'].options(transpose=True).value = list(range(10))

## Macros


## User defined functions (UDFs)

...

In [40]:
@xw.func
def greet_name(name):
    """Return a greeting to the inputted name."""
    return f'Hi, {name}'

## Limitations

While the link between Python and Excel provided by *xlwings* is very convenient, it has some limitations:

- Just like running macros in VBA, Ctrl-Z is not supported for undoing things. This can be quite frustrating.

## Other Python packages that interact with Excel

*xlwings* is only one of multiple packages in the Python ecosystem that can talk to Excel. Some other ones are *PyXLL* and *DataNitro*.

Furthermore, many packages provide a way to import and export data from Excel. Pandas for instance, can export a dataframes as a Excel file and import a range of cells into a dataframe. This is however quite different from actually having a "live" connection between Pytohn and Excel that can listen to individual cell values, run custom function etc.  

# Exercises

## Exercise 1 (Setup)

Perform the following tasks

1. Create an Excel/Python file pair by running `xlwings quickstart file_name` from the Prompt. See the section for Initial Setup above for more help. 


2. Create two new sheets called *Sheet2* and *Sheet3* in the workbook.


3. Download the *xlwings* add-in and load it into Excel. See the section for Initial Setup above for more help.


4. The created Python file already has some boiler plate code for importing *xlwings*, a `main()` function and a User Defined Function called `hello()`. 

   * Go ahead and delete the UDF `hello()` with its decorator. We are not going to use it here.

   * From the *xlwings* tab in the Excel ribbon, press the "Run main" button. This should run the `main()` function defined in the Python file.
   
It this succeeded, you should now see the text `'Hello World'` in Cell A1 in the first sheet of the workbook.


## Exercise 2

We don't have to use the `main()` function to export values from Excel to Python. You can also do this by running the Python code from the editor.

In the code block `if __name__ == '__main__':`, do the following:

1. Establish a connection to the workbook and save it as a variable, e.g. `wb`.
   
2. Create a variable for referring to *Sheet2*, e.g. `sht2`. 

3. Use NumPy's [arange](https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html) function to generate values from -50 to 50 with a step of 5 and insert the resulting 1D-array in cell *A1* in *Sheet2*.

***Hint:*** *Remember to use `.option(transpose=True)` if you want the values to be inserted vertically, your choice.*

***Recall:*** *The `if __name__ == '__main__'` block only gets executed when the Python file is run as a script, ***not*** if is is imported into another Python file. Pressing the "Run main" button executes the file directly (as a script).*


## Exercise 3

In this exercise, we will load a dataset and create a macro to interact with it.

1. **Download a csv file with weather data from Sydney**. You can find the file [here](https://github.com/Python-Crash-Course/Python201/blob/master/Session%207%20-%20Interaction%20with%20Excel/Sydney_weather.csv). Download it by choosing "Raw" => right-click => save as csv. 

    We are going to create a small interface for plotting the dataset. The user should be able to choose

  - Start date
  - End date
  - Parameter to plot
  
  In *sheet3* in the workbook, create input cells for the three things above.


2. Create a new function in the Python file. It should be placed *above* the `if __name__ == '__main__':` and have the `@xw.sub` decorator (signifying a macro). Just leave the function empty for now, we write the code in Step 4.

  After you have named your function in Python, go to Excel => Developer => Visual Basic => Module 1 and add the following VBA code to detect your Python function:
  
```vb
Sub {CHOOSE_MACRO_NAME}()
    mymodule = Left(ThisWorkbook.Name, (InStrRev(ThisWorkbook.Name, ".", -1, vbTextCompare) - 1))
    RunPython ("import " & mymodule & ";" & mymodule & ".{INSERT_PYTHON_FUNCTION_NAME}()")
End Sub
```

3. In Excel go to Insert => Shape and choose a shape for a button. Right-click the button and assign your macro to it.


4. Write the contents of the function in Step 2. The function has to:
  
   * Read the weather data file into a Pandas dataframe
   * Read the three input values from Excel
   * Filter the dataframe for all input parameters
   * Create a figure object and fill it with a plot of the filtered data
   * Put the resulting figure in *Sheet3* in the workbook
  
If everything is setup correctly, you should now be able to change the dates

---

***Hint:*** You can use this "helper" function for some of the grunt work. Note that only the macro function that in the end interacts with Excel needs to have the `@xw.sub` decorator. Note that you need to `import os` from the standard library to run the function.

```python
def create_daterange_dataframe(filename, start_date, end_date):

    # Get the 'correct' current working dir (where the .py and .xlsm files are)
    cwd = os.path.dirname(os.path.abspath(__file__))

    # Combine into a full path to the dataset file
    full_filename = f'{cwd}\\{filename}'

    df_raw = pd.read_csv(filename)

    # Change the "raw" dates into datetime objects so they can be filtered
    df_raw['Date'] = pd.to_datetime(df_raw['Date'])
    
    # Set the date as the index of the dataframe
    df = df_raw.set_index('Date')

    # Filter dataframe for input date range and return it
    return df[start_date:end_date]
```


***Explanation of the "path" above:*** *When running the Python macro from Excel, the ***current working directory*** might not be set correctly. This might be due to Excel dictating it instead of Python as normally. If this is the case, reading the data file will throw an import error if it is saved in the same directory as you `.xlsm` and `.py` files. The safest bet is to provide the full path of the csv-file. It could be done by this code:*


## Some improvements

Some ideas for improving our little weather plotting app cold be:

- Give the user a more helpful error message if the chosen `parameter` is not present as a column in the dataset. Currently, the dataset cannot be seen from inside Excel, so the user kind of has to "guess" the right string to put in. The default Pandas error is something like `"Keyerror: ..."`.


- Even better than an error message: Read all column name from the dataset and create a drop down in Excel so only valid entries can be chosen.


- Make the plot prettier and more readable.


- Automatically update the plot when either parameter is changed to avoid having to press the button. If not this, at removing/emptying the plot if either parameter change would be good. This will prevent having the temporary "wrong plot" until the button is pressed.


- Every time the "update plot" button is pressed, the code reads in the data as a dataframe and sorts it based on the date range and plotting parameter. This could be improved in many ways, everything from caching previously computed results to using a small database. This will speed up the code a lot and will probably be necessary if the plot should auto-update or if the dataset was larger.

### Other Pandas ideas for interaction with Excel

Some other ideas to try for this dataset:

- Generate descriptive statistics for entire datafrmae: [pandas.DataFrame.describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html)

- Compute statistics over custom time intervals: [pandas.DataFrame.resample](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html)



# End of exercises
The cell below is for setting the style of this document. It's not part of the exercises.

In [1]:
# Apply css theme to notebook
from IPython.display import HTML
HTML('<style>{}</style>'.format(open('../css/cowi.css').read()))